In [9]:
from utils import list_devices_memory

list_devices_memory()

GPU 0: Free 12537.19 MB / 81155.75 MB
GPU 1: Free 45157.19 MB / 81155.75 MB
GPU 2: Free 45015.19 MB / 81155.75 MB
GPU 3: Free 45159.19 MB / 81155.75 MB
GPU 4: Free 60155.19 MB / 81155.75 MB
GPU 5: Free 60035.19 MB / 81155.75 MB
GPU 6: Free 58515.62 MB / 81155.75 MB
GPU 7: Free 57025.62 MB / 81155.75 MB


In [12]:
import os

# Set the CUDA_VISIBLE_DEVICES environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Adjust the device IDs as needed

# Verify the setting
print("CUDA_VISIBLE_DEVICES:", os.environ["CUDA_VISIBLE_DEVICES"])

CUDA_VISIBLE_DEVICES: 1


In [ ]:
import lighteval
from datetime import timedelta
from lighteval.logging.evaluation_tracker import EvaluationTracker
from lighteval.models.vllm.vllm_model import VLLMModelConfig
from lighteval.pipeline import ParallelismManager, Pipeline, PipelineParameters
from lighteval.utils.utils import EnvConfig
from lighteval.utils.imports import is_accelerate_available

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use only GPU 0

if is_accelerate_available():
    from accelerate import Accelerator, InitProcessGroupKwargs
    accelerator = Accelerator(kwargs_handlers=[InitProcessGroupKwargs(timeout=timedelta(seconds=3000))], )
else:
    accelerator = None

evaluation_tracker = EvaluationTracker(
    output_dir="./results",
    save_details=True,
    push_to_hub=False,
    hub_results_org="your user name",
)

pipeline_params = PipelineParameters(
    launcher_type=ParallelismManager.ACCELERATE,
    env_config=EnvConfig(cache_dir="tmp/"),
    # Remove the 2 parameters below once your configuration is tested
    override_batch_size=1,
    max_samples=10,
)

model_config = VLLMModelConfig(
        pretrained="HuggingFaceTB/SmolLM-135M",
        dtype="float32",
        use_chat_template=True,
)

task = "helm|mmlu|5what |1"
# category | task | # few shot | # repeats

pipeline = Pipeline(
    tasks=task,
    pipeline_parameters=pipeline_params,
    evaluation_tracker=evaluation_tracker,
    model_config=model_config,
)

pipeline.evaluate()
# pipeline.save_and_push_results()
pipeline.show_results()



/opt/common/envs/rd/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-14 22:25:32,005	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Detected kernel version 5.4.286, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
--max_samples WAS SET. THESE NUMBERS ARE ONLY PARTIAL AND SHOULD NOT BE USED FOR COMPARISON UNLESS YOU KNOW WHAT YOU ARE DOING.
Detected kernel version 5.4.286, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/opt/common/envs/rd/lib/python3.11/site-packages/xformers/ops/fm

|                     Task                      |Version|Metric|Value |   |Stderr|
|-----------------------------------------------|------:|------|-----:|---|-----:|
|all                                            |       |em    |0.0018|±  |0.0018|
|                                               |       |qem   |0.0018|±  |0.0018|
|                                               |       |pem   |0.2579|±  |0.1362|
|                                               |       |pqem  |0.4333|±  |0.1581|
|helm:mmlu:_average:5                           |       |em    |0.0018|±  |0.0018|
|                                               |       |qem   |0.0018|±  |0.0018|
|                                               |       |pem   |0.2579|±  |0.1362|
|                                               |       |pqem  |0.4333|±  |0.1581|
|helm:mmlu:abstract_algebra:5                   |      0|em    |0.0000|±  |0.0000|
|                                               |       |qem   |0.0000|±  |0.0000|
|   